In [14]:
#Importing the necessary libraries
import time
import pandas as pd
import nltk
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [23]:
url = "https://www.daraz.pk/"

In [24]:
driver.get(url)

In [25]:
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Redmi 10 C' + Keys.RETURN)

In [ ]:
Grids = driver.find_elements(By.CLASS_NAME,"box--ujueT")
for Grid in Grids:
    Small_Grids = Grid.find_elements(By.CLASS_NAME,"gridItem--Yd0sa")

Data = {
    "Product Title": {},
    "Price": {},
    "Rating": {},
    "Rate": {},
    "Seller Rating": {},
    "Specifications": {},
    "Link" : {}
}

Review1 = []
i = 1
while(i<4):
    for i,Small_Grid in enumerate(Small_Grids):
        Link = Small_Grid.find_element(By.TAG_NAME,"a").get_attribute("href")
        time.sleep(2)
        driver.execute_script(f"window.open('{Link}', '_blank');")
        time.sleep(2)
        handles = driver.window_handles
        new_window = handles[-1]
        driver.switch_to.window(new_window)
        Data["Product Title"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span").text
        Data["Price"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span").text
        Data["Rating"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a").text
        try:
            Data["Rate"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span").text
        except NoSuchElementException:
            Data["Rate"][i] = 0
        Data["Seller Rating"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]").text
        Data["Specifications"][i] = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div[1]/div[1]/ul/li").text
        Data["Link"][i] = Link
        driver.execute_script("window.scrollTo(0, 500);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(500, 800);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(800, 1000);")
        time.sleep(2)
        driver.execute_script("window.scrollTo(1000, 1300);")
        time.sleep(2)
        try:
            Reviews = driver.find_elements(By.CLASS_NAME, "review-content")
            product_reviews = []
            for Review in Reviews:
                Review_container = Review.find_elements(By.CLASS_NAME, "review-item")
                for x in Review_container:
                    try:
                        data = x.find_element(By.CLASS_NAME, "review-content-sl").text
                        product_reviews.append(data)
                    except NoSuchElementException:
                        product_reviews.append("No Reviews")
        except NoSuchElementException:
            product_reviews.append("No Reviews")
        Review1.append(product_reviews)
        driver.close()
        original_window = handles[0]
        driver.switch_to.window(original_window)
        
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(2)

    next_page_button = driver.find_element(By.XPATH, "/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div[2]/div/ul/li[9]/a")
    next_page_button.click()
    i= i+1

   
    time.sleep(2)   
    

In [ ]:
driver.quit()

In [ ]:
df = pd.DataFrame(Data)
df

In [ ]:
df1 = pd.DataFrame(Review1)
df1

In [ ]:
result = pd.concat([df, df1], axis=1)
result

In [63]:
result.to_csv("New_Data7.csv")